### Enoncé

Le jeu de données contenu dans data.csv décrit des candidatures au poste de chercheur d’or chez OrFée. Votre objectif consiste à prédire le succès ou l’échec d’une candidature.
Le jeu de données comporte 11 colonnes :
- date – date de la candidature
- age – âge du candidat
- diplome – plus haut diplôme obtenu (bac, licence, master, doctorat)
- specialite – spécialité du diplôme (géologie, forage, détective, archéologie, . . . )
- salaire – salaire demandé
- dispo – oui : disponibilité immédiate, non : pas disponible immédiatement
- sexe – féminin (F) ou masculin (M)
- exp – nombre d’années d’expérience
- cheveux – couleur des cheveux (châtain, brun, blond, roux)
- note – note (sur 100) obtenue à l’exercice de recherche d’or
- embauche – le candidat a-t-il été embauché ? (0 : non, 1 : oui)

#### Version de python

In [ ]:
import platform
print(f"Python version : {platform.python_version()}")

#### Chargement des librairies

In [ ]:
import os
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

import pandas as pd
import numpy as np
from typing import Tuple, Union
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import scipy
from sklearn.preprocessing import LabelEncoder,StandardScaler

#### Seaborn charte graphique

In [ ]:
sns_colors = ["#9b59b6", "#3498db", "#95a5a6", "#e74c3c", "#34495e", "#2ecc71"]
sns_palette = sns.color_palette(sns_colors)
sns.set_style("darkgrid")
cm = sns.light_palette("green", as_cmap=True)

sns.palplot(sns_palette)
sns.set_palette(sns_palette)

In [ ]:
#Settings display pandas
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
#Matrice de corrélation Pearson
def corr_matrix(df: pd.DataFrame):
    plt.figure(figsize=(10,10))
    sns.heatmap(df.corr(),
                cmap=plt.cm.RdBu,
                vmax=1.0,
                linewidths=0.1,
                linecolor='white',
                square=True,
                annot=True
                )

In [ ]:
#Read data
data_orphee = pd.read_csv('../input/applicants-for-a-gold-digger-position/data.csv')
data_orphee.head(10)

## I. Statistiques descriptives

Jeu de données comportant 20 000 candidatures entre le 1er Janvier 2010 et le 31 décembre 2014.

In [ ]:
data_orphee.shape

Premières remarques
- date : colonne de type object à convertir en date et possibilité d'extraire les jours, mois et années de celle-ci
- cheveux : 4 types de cheveux, châtain étant le plus fréquent
- age : La moyenne d'âge des candidats au poste de chercheur d'or est de 35 ans. L’âge minimum d'un des candidats est de "-3 ans" ce qui constitue une valeur aberrante, à voir par la suite.
- exp : L'expérience moyenne est de 9 ans. exp min est égale à -2, à approfondir. A convertir en int
- salaire : Le salaire moyen est d'environ 35K
- sexe : Présence masculine majoritaire 
- diplôme : 4 types de diplôme, les candidats ont majoritairement un master
- spécialité : 4 types de spécialités, la majorité est spécialisée en géologie
- note : La note moyenne est de 75 ce qui laisse entendre que les candidats ont plutôt bien réussi l'exercice. Cependant la notation est /100, pourtant un candidat a eu la note de 143 ==> valeur aberrante à approfondir
- dispo : 2 types de disponibilités, la majorité des candidats ne sont pas disponibles immédiatement
- embauche : La variable cible que nous souhaitons prédire. A convertir en catégorielle.

Toutes les variables sauf embauche (variable cible) comportent des valeurs manquantes

In [ ]:
data_orphee.info()

In [ ]:
data_orphee.isnull().sum()

In [ ]:
#Colonnes à exclure Unnamed: 0 et index
data_orphee.describe(include='all')

In [ ]:
def preprocess(df):
    '''
        Phase de pre-processing : suppression, conversion de typologie de données ...
    '''
    col_to_drop = ['Unnamed: 0']
    df.drop(col_to_drop,axis=1,inplace=True)
    df['exp'] = pd.to_numeric(df['exp'],downcast='integer',errors='coerce')
    df['note'] = pd.to_numeric(df['note'],downcast='integer',errors='coerce')
    df['embauche'] = df['embauche'].astype('object')
    
    
    return df

In [ ]:
#Preprocessing des données
data_preprocess = preprocess(data_orphee)

89% des candidats n'ont pas été embauché tandis que 11% l'ont été. Ce qui montre que le poste de chercheur d'or chez OrFée a été très sélectif entre 2010 et 2014. </br>
De plus, cette information nous sera utile dans la réalisation du modèle permettant de prédire l'embauche d'un candidat (jeu de données non balancé)

In [ ]:
def display_countplot(df: pd.DataFrame,col_x: str,titre: str,hue: Union[None,str]=None):
    '''
        Affichage du graphe permettant de dénombrer la parts de candidats de chaque catégorie d'une variable catégorielle
    '''
    temp = df.copy()
    nb_rows = temp.shape[0]
    
    if col_x=='embauche' or hue=='embauche':
        temp['embauche'] = np.where(temp['embauche']==0,'non','oui')
    
    fig = plt.figure(figsize=(11,6))
    
    # plot grouped bar chart
    if hue is None:
        splot = sns.countplot(temp[col_x],palette=sns_palette)
    else:
        splot = sns.countplot(temp[col_x],hue=temp[hue],palette=sns_palette)
    splot.set_title(titre,fontsize=20)

    #Affichage des proportions au dessus des barplots
    for p in splot.patches:
        splot.annotate(format((p.get_height()/nb_rows), '.4f'),
                       (p.get_x()+ p.get_width()/ 2., p.get_height()),
                       ha = 'center',
                       va = 'center',
                       xytext = (0, 7),
                       textcoords = 'offset points')

In [ ]:
display_countplot(data_preprocess,'embauche',"Proportion d'embauches parmis les 20k candidatures")

Analysons les distributions : 
- Elles ont toute l'air d'être sans asymétrie prononcée ce qui est assez rare pour ce type de variables continues.
- Cependant en regardant les boîtes à moustaches on s'aperçoit qu'il y a de nombreuses valeurs statistiquement considérées comme des valeurs extrêmes donc aberrantes par rapport à la distribution de chacune de nos variables. 
- exemple : deux candidats ou plus ont une expérience inférieure à 0. Il en est de même pour l’âge. Ces valeurs seront à supprimer car elles risqueraient de biaiser l'apprentissage ainsi que les résultats du modèle

In [ ]:
def selectKindFeatures(df: pd.DataFrame,
                       kind: Union[None,str]='number')-> Union[str,pd.DataFrame]:
    '''
        Permet de sélectionner un type de données à partir d'un dataframe
    '''
    if len(df.select_dtypes(include=kind).columns) == 0:
        return f"Le dataframe ne contient pas de variables {kind}"
    return df.select_dtypes(include=kind)

In [ ]:
num_df = selectKindFeatures(data_preprocess)

In [ ]:
def display_distribution(df: pd.DataFrame,
                         numerical_cols: list):
    
    '''
        Affichage de la distribution des variables numériques
    '''

    plt.figure(figsize=(20,8))
    i = 0
    for column_ in numerical_cols:
        i = i+ 1
        plt.subplot(2, 4, i)
        sns.distplot(df[column_], label='Data', hist=False)
        plt.legend()
        plt.xlabel(numerical_cols[i-1], fontsize=12)

In [ ]:
num_cols = list(num_df.columns)
display_distribution(num_df,num_cols)

In [ ]:
def display_boxplot(df: pd.DataFrame,
                    numerical_cols: list):
    
    '''
        Affichage des boxplot des variables numériques
    '''
    plt.figure(figsize=(20,8))
    i = 0
    for columns_ in numerical_cols:
        i = i+1
        plt.subplot(2, 4, i)
        box = sns.boxplot(x=df[columns_], data=df, palette=sns_palette)
        #box.set_title(str(columns_),fontsize=15)

In [ ]:
display_boxplot(num_df,num_cols)

Cette fonction met en évidence les corrélations positives/négatives ou inexistantes entre les variables numériques et affiche leurs distributions différenciées par les candidats embauchés et ceux non embauchés. </br>
On aurait pu imaginer une corrélation positive entre l'âge et l'expérience ou entre le salaire et l'expérience ou encore entre les notes et l'expérience. 
Cependant avec surprise on remarque qu'il n'y a pas forcément de corrélation entre ces variables. On note juste visuellement une légère corrélation négative entre les notes et le salaire, ce qui peut être surprenant.

In [ ]:
def pair_plot(df: pd.DataFrame,
              numerical_cols: list,
              target: Union[None,str]=None):
    '''
        Affichage des corrélations deux à deux des variables numériques différencié par la variable embauche
    '''
    plt.figure(figsize=(10,10))
    if target is not None:
        numerical_cols.append(target)
        sns.pairplot(df[numerical_cols],height = 2, hue=target,palette=sns_palette)
        numerical_cols.remove(target)
    else:
        sns.pairplot(df[numerical_cols],height = 2,palette=sns_palette)

In [ ]:
pair_plot(data_preprocess,num_cols,target='embauche')

La droite de régression nous montre le sens par lequelle le plus de points passent. </br>
Cependant on voit qu'ils sont éparpillés, ce qui prouve que la dépendance est très légère

In [ ]:
sns.jointplot("note","salaire",data=num_df,kind='reg')

### II. Exploration et Transformation des données

In [ ]:
def extract_features_from_dates(df: pd.DataFrame):
    '''
        Extraction du jour, mois et année de la date
    '''
    df['jour']=(df['date'].apply(lambda x: x.day)).astype('object')
    df['mois']=(df['date'].apply(lambda x: x.month)).astype('object')
    df['annee']=(df['date'].apply(lambda x: x.year)).astype('object')
    
    return df

def transfo(df: pd.DataFrame):
    '''
        Toute transformation de données ou ajouts de features 
    '''
    df['date'] = pd.to_datetime(df['date'])
    df = extract_features_from_dates(df)
    niveau_diplome = {'bac':0,'licence':1,'master':2,'doctorat':4}
    df['niveau_diplome'] = df['diplome'].map(niveau_diplome)
    
    return df

In [ ]:
#Application de la fonction transfo qui a pour but de rajouter de la feature aux données existantes
data_transfo = transfo(data_preprocess)

In [ ]:
data_transfo.shape

#### Intéressons nous aux valeurs aberrantes

Trouvons les valeurs aberrantes par conditions :
1) Candidats ayant une expérience supérieure à leurs âges </br>
2) Candidats ayant une note inférieure à 0 ou supérieur à 100 </br>
3) Candidats ayant une expérience inférieure à 0 </br>
4) Candidats ayant un niveau de diplôme supérieur ou égale au bac tandis qu'ils ont au plus 16 ans </br>

In [ ]:
#Liste des conditions ordonnées en fonction du listing fait ci-dessus
conditions = [(data_transfo.exp > data_transfo.age),
              (np.logical_or(data_transfo.note < 0,data_transfo.note > 100)),
              (data_transfo.exp < 0),
             (np.logical_and(data_transfo.niveau_diplome >=0 , data_transfo.age <=16))]

In [ ]:
#Boucle permettant d'afficher le nombre de candidats concernés par chacune des conditions
#Ainsi que le nombre d'embauchés parmis eux
i = 0
for cond in conditions:
    i = i+1
    index_to_delete = data_transfo[cond].index
    nb_pers = len(data_transfo[cond])
    try:
        nb_embauche = data_transfo[cond].embauche.value_counts()[1]
        print(f"{nb_pers} candidats sont concernés par la condition n°{i} dont {nb_embauche} embauchés")
    except:
        print(f"{nb_pers} candidats sont concernés par la condition n°{i} dont 0 embauchés")
    # Suppression des candidats respectant ces conditions
    data_transfo.drop(index_to_delete,axis=0,inplace=True)
data_transfo.drop(['niveau_diplome'],axis=1,inplace=True)

In [ ]:
data_transfo.shape

##### Ce traitement nous a permis d'évincer 1 998 lignes, ce qui a permis de gagner environ 1% de représentativité de nos candidats embauchés soit environ 12%

#### Intéressons nous aux valeurs manquantes

Pour ce faire, nous allons utiliser des fonctions personnalisées ainsi que la librairie missngno qui permet d'analyser les valeurs manquantes. </br>
- Diagramme en bâton : Ce graphe nous montre le nombre de valeurs manquantes par colonnes. On remarque toutefois qu'il n'y a pas beaucoup de valeurs manquantes.
- Matrice de corrélation : On aperçoit aucune corrélation quant à la présence ou non de valeurs manquantes. Seuls les variables mois, année et jour sont corrélées car elles ont été construite à partir de la même variable :  Date

In [ ]:
def missingValues(df: pd.DataFrame):
    null_df = pd.DataFrame((df.isnull().sum().sort_values(ascending = False)),columns=['null_values'])
    null_df = null_df[null_df.null_values>0]
    plt.figure(figsize=(10,5))
    bar = sns.barplot(null_df.index,null_df.null_values,palette=sns_palette)
    bar.set_title("Nombre de valeurs manquantes par colonnes",fontsize=16)
    
    return list(null_df.index)

In [ ]:
nulls_cols = missingValues(data_transfo)

In [ ]:
import missingno as msno
missingdata_df = data_transfo.columns[data_transfo.isnull().any()].tolist()

In [ ]:
msno.heatmap(data_transfo[missingdata_df], figsize=(10,5))

##### Y a t-il une corrélation entre les valeurs manquantes et la variable cible ? </br>
Pour y répondre nous allons voir si parmi les lignes présentant des valeurs manquantes, il y a un pattern qui permet de savoir si un candidat est embauché ou non. </br>
Nous vérifions cela avec une règle simpliste : Vérifier la présence unique de candidats embauchés ou non dès lors qu'une variable contenant des valeurs manquantes apparait

In [ ]:
def analyse_missing_data(df: pd.DataFrame,
                         null_cols: list,
                         target: str):
    for col in null_cols:
        if df[df[col].isnull()][target].value_counts()[0] == 0 or df[df[col].isnull()][target].value_counts()[1] == 0:
            print(f"Les valeurs manquantes de la colonne {col} ont peut-être un lien avec la colonne {target}")
        else:
            print(f"Les valeurs manquantes de la colonne {col} n'influent pas sur la variable cible : {target}")

In [ ]:
# Nous venons de vérifier qu'aucune valeurs manquantes n'explique la variable cible
analyse_missing_data(data_preprocess,null_cols=nulls_cols,target='embauche')

##### Conclusion sur les valeurs manquantes : 
Les variables présentant des valeurs manquantes ne sont pas expliquées par une quelconque corrélation avec une autre variable ni avec la variable cible. On peut en déduire qu'elles sont réparties de manière aléatoire. </br>
Des méthodes d'imputation de valeurs manquantes basiques peuvent être mises en place. </br> 
C'est à dire remplacer ces valeurs par la moyenne, la médiane ou le mode pour les variables catégorielles. </br>
Cependant, la méthode choisie dans notre cas sera de supprimer ces candidats de notre jeu de données car ils n'engendrent pas un plus grand écart entre nos deux classes.

In [ ]:
data_transfo = data_transfo.dropna(axis=0)

In [ ]:
data_transfo.shape

##### Ce traitement nous a permis d'évincer 896 lignes, ce qui laisse intact la représentativité de nos candidats embauchés soit environ 12%

#### Intéressons nous maintenant aux valeurs aberrantes expliquées de manière statistique

Plusieurs méthodes peuvent être utilisées afin de repérer des valeurs aberrantes. Le contexte n'étant pas critique, nous allons faire au plus simple, c’est à dire supprimer ces valeurs en nous aidant des quantiles. </br>
L'idée ici est de calculer l'intervalle interquartile de chacune des variables numériques et de supprimer chaque point qui serait en dessous de ce que nous avons nommé IQR_min et les valeurs supérieur à IQR_max

In [ ]:
def compute_IQR_find_outliers(df: pd.DataFrame,cols: list):
    '''
        Calcul de l'interval interquartile et suppression des valeurs extreme
    '''
    Q1 = df[cols].quantile(0.25)
    Q3 = df[cols].quantile(0.75)
    IQR = Q3 - Q1
    IQR_min = Q1 - 1.5 * IQR
    IQR_max = Q3 + 1.5 * IQR
    
    df_out = df[~((df[cols] < IQR_min) | (df[cols] > IQR_max)).any(axis=1)]
    
    return df_out

In [ ]:
data = compute_IQR_find_outliers(data_transfo,num_cols)

In [ ]:
# 291 valeurs étaient des valeurs extrêmes 
data.shape

##### Ce traitement nous a permis d'évincer 291 lignes, ce qui laisse intact la représentativité de nos candidats embauchés soit à environ 12%

In [ ]:
#Les boites à moustaches sont plus "propres" c'est à dire qu'elles n'ont plus de valeurs extrêmes
# d'autant plus que leurs asymétries n'a visuellement pas changé.
display_boxplot(data,num_cols)

In [ ]:
# Afin d'y voir plus clair dans nos graphes, je partage en trois quantiles les variables note et salaire qui possède de nombreuses valeurs uniques
data['note_cut']=pd.qcut(data['note'],q=3,labels=['Low','Mid','High'])
data['salaire_cut']=pd.qcut(data['salaire'],q=3,labels=['Low','Mid','High'])

In [ ]:
# Description des groupes note
data.groupby('note_cut')['note'].describe()

In [ ]:
# Description des groupes salaire
data.groupby('salaire_cut')['salaire'].describe()

#### Nous allons maintenant essayé de voir si nos variables numériques permettent de différencier un candidat embauché d'un autre non embauché </br>

##### Tendances générales :
- Boites à moustaches : Il est compliqué de voir des tendances se former à partir de ces graphes. Cela prouve que nos variables numériques ne permettent pas en elles-mêmes de différencier un candidat embauché d'un non embauché. Cependant, nous avons remarqué quelques légères différences grâces aux variables d'expérience ou de salaire. Celles-ci nous montrent que les candidats embauchés ont une très légère tendance à avoir des valeurs moins élevées que les candidats non embauchés. Mais aucune conclusion ne peut être faite à ce niveau.
- Diagramme en bâton : L'idée ici est de considérer notre variable embauché/ou non embauché comme une variable numérique (1 ou 0). Ainsi, de calculer le "taux d'embauche" par variable. Exemple : sur les 16k candidats 172 ont 19 ans, parmi ceux-là, 153 sont non embauchés et 19 le sont (153*0+19*1) /153+19 = 0.110. Ceci ne sert qu’à avoir une tendance des catégories ou il y a le plus d’embauches.
</br></br>
Conclusion : Nous arrivons mieux à distinguer des tendances :  Les candidats demandant un salaire de 30K en moyenne ; Ceux possédant des notes tournant autour d’environ 73/100 ou encore ceux ayant le plus d'expérience sont le plus embauchés. Cependant, en ce qui concerne l'âge, cela varie beaucoup trop pour en déduire une quelconque tendance.

In [ ]:
def display_boxplot_differenciate(df: pd.DataFrame,
                    numerical_cols: list,
                    target: str):
    temp = df.copy()
    plt.figure(figsize=(18,6))
    i = 0
    if target=='embauche':
        temp['embauche'] = np.where(temp['embauche']==0,'non','oui')
    for columns_ in numerical_cols:
        i = i+1
        plt.subplot(1, 4, i)
        box = sns.boxplot(x=temp[target], y=temp[columns_], data=temp, palette = sns_palette)
        box.set_title(f"{columns_} vs {target}",fontsize=15)

In [ ]:
display_boxplot_differenciate(data,num_cols,"embauche")

In [ ]:
def display_barplot_differenciate(df: pd.DataFrame,
                    numerical_cols: list,
                    target: str):
    temp = df.copy()
    plt.figure(figsize=(23,8))
    i = 0
    temp.embauche  = temp.embauche.astype("int")
    for columns_ in numerical_cols:
        i = i+1
        if ("age") in numerical_cols:
            plt.subplot(1, 1, i)
        else:
            plt.subplot(2, 3, i)
        df_grouped = temp.groupby(columns_)[target].mean().reset_index()
        df_grouped = df_grouped[df_grouped[target]>0]
        bar = sns.barplot(x=df_grouped[columns_], y=df_grouped[target], data=df_grouped, palette = sns_palette)
        bar.set_title(f"{columns_} vs {target}",fontsize=15)

In [ ]:
cols = ["salaire_cut","note_cut","exp"]
display_barplot_differenciate(data,numerical_cols=cols,target="embauche")

In [ ]:
cols = ["age"]
display_barplot_differenciate(data,cols,"embauche")

#### Intéressons nous aux variables catégorielles

In [ ]:
df_cat = selectKindFeatures(data,kind='object')

In [ ]:
df_cat.head()

In [ ]:
def cramersv_corr(x: pd.Series, y: pd.Series):
    '''
        Calcule de la corrélation de cramers V entre deux variables catégorielles
    '''
    confusion_matrix = pd.crosstab(x,y)
    chi2 = scipy.stats.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2-((k-1)*(r-1))/(n-1))
    rcorr = r-((r-1)**2)/(n-1)
    kcorr = k-((k-1)**2)/(n-1)
    return np.sqrt(phi2corr/min((kcorr-1),(rcorr-1)))

def cramer_matrix(categoricals: pd.DataFrame) -> pd.DataFrame:
    """
        Fonction permettant de construire une matrice à partir de du calcul des coefs de correlation de cramers V
    """
    categ_col_name = categoricals.columns
    n_var = len(categ_col_name)
    correlation_matrix = pd.DataFrame(np.ones((n_var, n_var)),
                                      index=categ_col_name,
                                      columns=categ_col_name)
    for name1, name2 in itertools.combinations(categ_col_name, 2):
        if name1 == name2:
            cramer_coef = 1
        else:
            cramer_coef = cramersv_corr(categoricals[name1],
                                                 categoricals[name2])
        correlation_matrix.loc[name1, name2] = cramer_coef
        correlation_matrix.loc[name2, name1] = cramer_coef
    return correlation_matrix

Le test de corrélation de cramer V nous montre l'intensité dans la relation entre deux variables catégorielles. </br>
Remarques : 
- Nous observons une très forte relation entre la disponibilité et le diplôme. Les candidats ayants un master ou un doctorat sont plus souvent disponibles immédiatement que les candidats ayant une licence ou un bac. Ce qui fait sens. 
- Il y a une forte relation entre la disponibilité et la spécialité. Les archéologues ou les détectives sont le plus souvent disponibles.

In [ ]:
coor_mat = cramer_matrix(df_cat)
cm = sns.light_palette("green", as_cmap=True)
coor_mat.style.background_gradient(cmap=cm)

In [ ]:
display_countplot(df_cat,'diplome',"Diplome vs Disponibilité",hue='dispo')

In [ ]:
display_countplot(df_cat,'specialite',"Specialité vs Disponibilité")

In [ ]:
display_countplot(df_cat,'annee',"Année vs Embauche",hue='embauche')

In [ ]:
display_countplot(df_cat,'diplome',"Diplôme vs Cheveux",hue='cheveux')

#### Y a-t-il une dépendance statistiquement significative entre la spécialité et le sexe ?

La majorité des candidats au poste de chercheur d'or sont des hommes

In [ ]:
display_countplot(data_preprocess,'sexe',"Proportion d'hommes et de femmes parmi les candidats")

La specialité la plus représentée parmi les candidats est la géologie

In [ ]:
display_countplot(data_preprocess,'specialite',"Proportion par specialité des candidats")

Les specialités géologie et forage sont le plus représentées par des hommes tandis que détective et archeologie par des femmes 

In [ ]:
display_countplot(data_preprocess,'specialite',"Specialite vs Sexe",hue="sexe")

le coefficient de cramer montre une forte relation entre la specialité et le sexe

In [ ]:
cramersv_corr(df_cat['specialite'],df_cat['sexe'])

Test du chi2 carré de Pearson: </br>
Ce test d'hypothèse permet de vérifier s'il existe une association significative entre deux variables catégorielles dans un tableau de contingence. </br>
Ici nous avons fait le calcul, et nous trouvons que la p-value est inférieure à 0.05 donc les variables ne sont pas indépendantes. L'hypothèse nulle est donc rejetée.

In [ ]:
ct = pd.crosstab(df_cat['sexe'],df_cat['specialite'])
ct

In [ ]:
chi2,p,dof,expected = scipy.stats.chi2_contingency(ct)

In [ ]:
print(f"p-value: {p} inférieure au seuil de significativité fixé à 0.05")

#### Y a-t-il une dépendance statistiquement significative entre La couleur de cheveux et le salaire demandé?

Observons le salaire demandé par les candidats et leurs couleurs de cheveux. Nous remarquons que la différence est légère, les candidats aux cheveux châtain auraient des prétentions salariale plus élevées tandis que les candidats aux cheveux roux auraient attentes moindres. Cependant cela peut s'expliquer par la faible representativité des candidats aux cheveux roux parmi les candidats

In [ ]:
plt.figure(figsize=(15,9))
box = sns.boxplot("cheveux","salaire",data=data,palette=sns_palette)
box.set_title('Cheveux vs salaire',fontsize=20)

Si nous affichons leurs distributions nous remarquons que la différence est très légère

In [ ]:
data.groupby("cheveux")["salaire"].agg([np.mean, np.std, np.median, np.min, np.max])

In [ ]:
plt.figure(figsize=(15,9))
sns.distplot( data.salaire[data.cheveux == "brun"], color="skyblue", label="Brun")
sns.distplot( data.salaire[data.cheveux == "blond"] , color="coral", label="Blond")
sns.distplot( data.salaire[data.cheveux == "roux"] , color="yellow", label="Roux")
sns.distplot( data.salaire[data.cheveux == "chatain"] , color="olivedrab", label="Chatain")
plt.legend()

Le tau de kendall permet de vérifier la corrélation entre une variable catégorielle et une numérique. </br>
Comme le confirme ce coefficient, il n'y a pas de lien statistiquement significatif entre la couleur des cheveux et le salaire.

In [ ]:
def kendalltau_corr(x, y):
    x_arr=np.array(pd.DataFrame(x))
    y_arr=np.array(pd.DataFrame(y))
    corr,_=scipy.stats.kendalltau(x_arr,y_arr)
    return corr

In [ ]:
kendalltau_corr(data["cheveux"],data["salaire"])

#### Y a-t-il une dépendance statistiquement significative entre le nombre d’années d’expérience et la note à l’exercice ?

In [ ]:
plt.figure(figsize=(15,9))
box = sns.countplot("exp",data=data,palette=sns_palette)
box.set_title('Expérience',fontsize=20)

In [ ]:
plt.figure(figsize=(15,9))
box = sns.boxplot("exp","note",data=data,palette=sns_palette)
box.set_title('Expérience vs note',fontsize=20)

In [ ]:
data.groupby("exp")["note"].agg([np.mean, np.std, np.median, np.min, np.max])

Le tau de kendall nous montre qu'il n'y a pas de lien statistiquement significatif entre l'expérience et la note. 

In [ ]:
kendalltau_corr(data["exp"],data["note"])

#### Preparation du dataframe pour l'application d'un modèle de machine learning

- Suppression des variables non utiles pour la prédiction
- Encodage des variables à valeurs binaires
- Conversion des variables categorielles en variables indicatrices (dummy variable)

In [ ]:
cols_to_delete = ['date','jour','note_cut','salaire_cut','mois','annee']
df_final = data.drop(cols_to_delete,axis=1)

In [ ]:
df_final.head()

In [ ]:
cols = ["cheveux","sexe","specialite","dispo","diplome"]
le = LabelEncoder()
cols_to_dummies = list()
for col in cols:
    if len(set(df_final[col]))>2:
        cols_to_dummies.append(col)
    else:
        df_final[col] = le.fit_transform(df_final[col])

In [ ]:
df_final = pd.get_dummies(data=df_final,columns=cols_to_dummies)

In [ ]:
cols_to_scale = ["age","exp","salaire","note"]
scaler = StandardScaler()
df_final[cols_to_scale] = scaler.fit_transform(df_final[cols_to_scale])

In [ ]:
df_final.head()

In [ ]:
temp = df_final.copy()
temp.embauche = temp.embauche.astype("int")
plt.figure(figsize=(10,10))
tc = temp.corr()
cm = sns.light_palette("green", as_cmap=True)
tc.style.background_gradient(cmap=cm)

A ce stade nous pouvons émettre l'hypothèse que les variables qui pourraient avoir une influence sur les prédictions de la variable cible sont : 
   - La spécialité
   - Le diplome
   - Le salaire

In [ ]:
df_final.to_csv('/kaggle/working/data_prepro.csv', encoding='utf-8',index=False)